In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose

    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)

    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        # weights initialization
        self.theta = np.zeros(X.shape[1])

        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient

            if(self.verbose == True and i % 10000 == 0):
                print(f'loss: {self.__loss(h, y)} \t')

    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)

        return self.__sigmoid(np.dot(X, self.theta))

    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold

# Ejemplo de uso:
# Supongamos que 'X' son las características y 'y' es la etiqueta objetivo
# X = np.array([[a, b], [c, d]])
# y = np.array([0, 1])
# model = LogisticRegression(lr=0.1, num_iter=300000)
# model.fit(X, y)
# preds = model.predict(X)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Cargamos el dataset
# Suponemos que tenemos un DataFrame 'df' con las columnas 'features' y 'target'
# X = df[['feature1', 'feature2']]  # por ejemplo
# y = df['target']

# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos el modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X_train, y_train)

# Realizamos predicciones
predictions = model.predict(X_test)

# Evaluamos el modelo
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy}')
